In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import numpy as np
import tabula
import re
import sqlite3
from Functions.refactor_dataframe_columns import * 

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13052\1440494663.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import os
allFilesinDirectory = os.listdir("../Inputs/")

In [4]:

usefulFiles = []
for k in allFilesinDirectory:
    if k.__contains__("3_1_C"):
        usefulFiles.append(k)
print(usefulFiles)

['DocEconomica_384_3_1_C_2020_12_31_00_00_00_000.pdf', 'DocEconomica_384_3_1_C_2021_12_31_00_00_00_000.pdf', 'DocEconomica_384_3_1_C_2022_12_31_00_00_00_000.pdf', 'DocEconomica_6712_3_1_C_2020_12_31_00_00_00_000.pdf', 'DocEconomica_6712_3_1_C_2021_12_31_00_00_00_000.pdf', 'DocEconomica_6712_3_1_C_2022_12_31_00_00_00_000.pdf', 'DocEconomica_736008_3_1_C_2021_12_31_00_00_00_000.pdf', 'DocEconomica_736008_3_1_C_2022_12_31_00_00_00_000.pdf']


In [5]:
route = "../Inputs/"
for k in range(len(usefulFiles)):
    usefulFiles[k] = route+usefulFiles[k]
print(usefulFiles)

['../Inputs/DocEconomica_384_3_1_C_2020_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_384_3_1_C_2021_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_384_3_1_C_2022_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_6712_3_1_C_2020_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_6712_3_1_C_2021_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_6712_3_1_C_2022_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_736008_3_1_C_2021_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_736008_3_1_C_2022_12_31_00_00_00_000.pdf']


In [6]:
#files = (glob.glob("../Inputs/*3_1_A*"))
#files.clear()
files = [
    "../Inputs/DocEconomica_384_3_1_A_2020_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_384_3_1_A_2021_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_384_3_1_A_2022_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_6712_3_1_A_2022_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_736008_3_1_A_2021_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_736008_3_1_A_2022_01_31_11_46_22_593.pdf",
    "../Inputs/DocEconomica_736008_3_1_A_2022_12_31_00_00_00_000.pdf"
]

In [7]:
dataPDF = pd.DataFrame()
sqliteFile = "../Outputs/datos_ERI2.db"
pos: int = 0        #SELECCIONA EL ARCHIVO Y SETEA FLAGS PARA EL RESTO DEL ARCHIVO
validFiles = [
    "ESTADO DE FLUJO EFECTIVO POR EL MÉTODO DIRECTO",        # SALDOSBALANCE EN USD 0
    "ESTADO DE SITUACIÓN FINANCIERA",                        # VALOR EN USD         1
    "ESTADO DE RESULTADO INTEGRAL"                           # VALOR EN USD         2
]
generalColumns = ["CUENTA",
                "VALOR EN USD",
                "SALDOSBALANCE EN USD"]
originalColumnsToReplace = [
                "SALDOS BALANCE (En USD$)",
                "VALOR (En USD$)"
]


tableNames = [
    "estado_flujo",                                         # 0
    "estado_financiera",                                    # 1
    "estado_integral"                                       # 2
]

fileType = validFiles[pos]#fileType = "ESTADO DE RESULTADO INTEGRAL"


In [8]:
try:
    for document in usefulFiles:
        dataPDF = tabula.read_pdf(document, pages="all", multiple_tables=True, lattice=True,
                                    output_format="dataframe", encoding="latin-1")[0]
        print(dataPDF[:20])
        listaColumnas = dataPDF.columns.to_list()

        rowStartIndex = dataPDF.index[ dataPDF[listaColumnas[0]]==fileType ].to_list()
        dfHeader = dataPDF.iloc[ : rowStartIndex[0]].copy()

        dataPDF.drop([listaColumnas[1]], axis=1, inplace=True)
        
        if pos == 1 or pos == 2:
            dataPDF.columns = [generalColumns[0],generalColumns[1]] #dataPDF.columns = ["CUENTA","VALOR EN USD"]
        else:
            #pos == 0
            dataPDF.columns = [generalColumns[0],generalColumns[2]]

        listaColumnas.clear()
        listaColumnas = dataPDF.columns.to_list()

        columnaOriginal = ""
        if pos == 1 or pos == 2:
            columnaOriginal = originalColumnsToReplace[1]
        else:
            #pos == 0
            columnaOriginal = originalColumnsToReplace[0]
            
        indexCodValor = dataPDF[ ( dataPDF[listaColumnas[0]]==listaColumnas[0] ) &
                ( dataPDF[listaColumnas[1]]==columnaOriginal )
            ].index
        
        dataPDF.drop(indexCodValor, inplace=True)



        patternDelNull = r"ull"
        filterNullStrings = dataPDF[ dataPDF[listaColumnas[0]].str.contains(patternDelNull, na=False) ].index
        dataPDF.drop(filterNullStrings, inplace=True)

        filterNullStrings = dataPDF[ ( dataPDF[listaColumnas[0]].isna() ) &
                ( dataPDF[listaColumnas[1]].isna() ) ].index
        
        dataPDF.drop(filterNullStrings, inplace=True)
        dataPDF.reset_index(drop=True, inplace=True)


        rowStartIndex = dataPDF.index[ dataPDF[listaColumnas[0]]=="REPRESENTANTE(S) LEGAL(ES)" ].to_list()
        dfContador = dataPDF.iloc[ rowStartIndex[0]: ].copy()
        dfContador.columns = ["REPRESENTANTE(S) LEGAL(ES)","CONTADOR"]


        filterNullStrings = dfContador[ 
            (dfContador[dfContador.columns.to_list()[0]]=="REPRESENTANTE(S) LEGAL(ES)") |
            (dfContador[dfContador.columns.to_list()[0]]=="CONTADOR") ].index
        dfContador.drop(filterNullStrings, inplace=True)
        
        listaColumnasContador = dfContador.columns.to_list()
        dfContador.reset_index(drop=True, inplace=True)

        pattern = dfContador.loc[0,"REPRESENTANTE(S) LEGAL(ES)"].split("\r")[0]
        similar_rows = dfContador[dfContador[listaColumnasContador[0]].str.contains(pattern, case=False, na=False)]
        if(len(similar_rows) > 1):
            dfContador.drop(similar_rows.index.to_list()[0], inplace=True)
        
        dfContador.reset_index(drop=True, inplace=True)
        ColumnasContador= ["REPRESENTANTE LEGAL","IDENT. REPRESENTANTE LEGAL","CONTADOR","IDENT. CONTADOR","COD.CONTADOR"]

        datosContador = []
        if len(dfContador) >= len(ColumnasContador):
            for k in range(len(ColumnasContador)):
                datosContador.append( dfContador.loc[k, listaColumnasContador[0]] )
        else:
            for k in range(len(ColumnasContador)):
                if k in range(len(dfContador)):
                    datosContador.append( dfContador.loc[k, listaColumnasContador[0]] )
                else:
                    datosContador.append(" ")
        
        headerColumns = dfHeader.columns.to_list()
        dfHeader.drop([headerColumns[-1]], axis=1, inplace=True)
        headerColumns.pop(0)
        dfHeader.columns = headerColumns

        newFirstRow = pd.DataFrame([dfHeader.columns.to_list()], columns=dfHeader.columns)
        dataFrameHeaderContador = pd.concat([newFirstRow, dfHeader])
        dataFrameHeaderContador.reset_index(drop=True, inplace=True)

        rowStartIndex = dataPDF.index[ dataPDF[listaColumnas[0]]==fileType ].to_list()
        dataPDF = dataPDF.iloc[ rowStartIndex[0]: ]
        dataPDF.reset_index(drop=True, inplace=True)

        rowStartIndex = dataPDF.index[ dataPDF[listaColumnas[0]]=="REPRESENTANTE(S) LEGAL(ES)" ].to_list()
        dataPDF = dataPDF.iloc[ : rowStartIndex[0] ]

        filterNullStrings = dataPDF[ (dataPDF[listaColumnas[0]]==fileType) & 
                (dataPDF[listaColumnas[1]].isna()) ].index
        dataPDF.drop(filterNullStrings, inplace=True)

        filterNullStrings = dataPDF[ (dataPDF[listaColumnas[0]].isna()) & 
                (dataPDF[listaColumnas[1]].isna()) ].index
        dataPDF.drop(filterNullStrings, inplace=True)
        dataPDF.reset_index(drop=True, inplace=True)

        if len(datosContador) <= 0:
            for k in range(len(ColumnasContador)):
                datosContador.append(" ")

        dfContador = pd.DataFrame([datosContador], columns =ColumnasContador)
        transposedHeader = dataFrameHeaderContador.set_index( dataFrameHeaderContador.columns.to_list()[0]).T


        for k in range( len(dfContador.columns.to_list()) ):
            transposedHeader.insert(k, dfContador.columns.to_list()[k], dfContador.loc[0, dfContador.columns.to_list()[k]])
        
        transposedHeader.reset_index(drop=True, inplace=True)
        transposedHeader.columns = [
            remove_accents_and_replace(col) for col in transposedHeader.columns
        ]


        transposedData = dataPDF.set_index( listaColumnas[0] ).T

        transposedData.columns = [
            remove_accents_and_replace(col) for col in transposedData.columns
        ]
        datosTransposed = rename_duplicate_columns(transposedData)

        data = remove_trailing_number_in_column(datosTransposed)

        #31/03/2024
        columnasTransposedHeader = transposedHeader.columns.to_list()
        expAnioIndex = columnasTransposedHeader.index("expediente")
        data.insert( 0,"expediente", transposedHeader.loc[0, columnasTransposedHeader[expAnioIndex]] )
        expAnioIndex = columnasTransposedHeader.index("anio")
        data.insert( 0,"anio", transposedHeader.loc[0, columnasTransposedHeader[expAnioIndex]] )

        conn = sqlite3.connect(sqliteFile)
        cursor = conn.cursor()

        chosenTable:str = ""
        if pos == 0:
            chosenTable = tableNames[pos]

        elif pos == 1:
            chosenTable = tableNames[pos]

        elif pos == 2:
            chosenTable = tableNames[pos]


        #cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='contabilidad'")
        cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{chosenTable}'")
        table_exists = cursor.fetchone()

        if not table_exists:
            cursor.execute(f"CREATE TABLE {chosenTable} (id INTEGER PRIMARY KEY)")
            #cursor.execute("CREATE TABLE contabilidad (id INTEGER PRIMARY KEY)")

        #cursor.execute("PRAGMA table_info(contabilidad)")
        cursor.execute(f"PRAGMA table_info({chosenTable})")
        existing_columns = [column[1] for column in cursor.fetchall()]

        new_columns = [column for column in datosTransposed.columns if column not in existing_columns]

        for column in new_columns:
            cursor.execute(f"ALTER TABLE {chosenTable} ADD COLUMN {column} NULL")
            #cursor.execute(f"ALTER TABLE contabilidad ADD COLUMN {column} NULL")
            
        conn.commit()

        #data.to_sql("contabilidad", conn, if_exists="append", index=False)
        data.to_sql(chosenTable, conn, if_exists="append", index=False)
        transposedHeader.to_sql("cabecera", conn, if_exists="append", index=False)
except Exception as e:
    print(f"Error: {e}")
    print(f"Archivo: {document}")

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


                                           Unnamed: 0  \
0                                           DIRECCIÓN   
1                                          EXPEDIENTE   
2                                                 RUC   
3                                                 AÑO   
4                                          FORMULARIO   
5      ESTADO DE FLUJO EFECTIVO POR EL MÉTODO DIRECTO   
6                                              CUENTA   
7   INCREMENTO NETO (DISMINUCIÓN) EN EL EFECTIVO Y...   
8   FLUJOS DE EFECTIVO PROCEDENTES DE (UTILIZADOS ...   
9       CLASES DE COBROS POR ACTIVIDADES DE OPERACIÓN   
10  COBROS PROCEDENTES DE LAS VENTAS DE BIENES Y P...   
11  COBROS PROCEDENTES DE REGALÍAS, CUOTAS, COMISI...   
12  COBROS PROCEDENTES DE CONTRATOS MANTENIDOS CON...   
13  COBROS PROCEDENTES DE PRIMAS Y PRESTACIONES, A...   
14          OTROS COBROS POR ACTIVIDADES DE OPERACIÓN   
15        CLASES DE PAGOS POR ACTVIDADES DE OPERACIÓN   
16  PAGOS A PROVEEDORES POR EL 